##### Script to analayze website data for the Case Study 1 of the ATPA modules

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import openpyxl


In [89]:
tracking = pd.read_csv("Data/tracking.csv").sort_values(by=['Day'])
print('tracking')
display(tracking.head())
print('Shape of the daily DataFrame is {}'.format(tracking.shape))

daily = pd.concat([pd.read_excel("Data/site_daily.xlsx", sheet_name='First Half'), 
                   pd.read_excel("Data/site_daily.xlsx", sheet_name='Second Half')], 
                   axis=0)
print('\n daily')
display(daily.head())
print('Shape of the daily DataFrame is {}'.format(daily.shape))
#### Variables:
# date - values from Jan 1 - Dec 31, 2017
# news - 1, if news story posted; 0, else
# num_ads - number of ads posted on the web page
# num_links - number of links to articles
# headliner_topic - topic of the top article linked


tracking


,Day,Duration,Previous,gender
110,2017-03-15,1,0,Female
141,2017-03-15,3,0,Unlisted
142,2017-03-15,1,1,Male
143,2017-03-15,2,1,Male
144,2017-03-15,6,1,Female


Shape of the daily DataFrame is (22406, 4)

 daily


,date,news,num_ads,num_links,headliner_topic
0,2017-01-01,1.0,5,3,technology
1,2017-01-02,1.0,5,1,technology
2,2017-01-03,1.0,5,4,visualization
3,2017-01-04,0.0,6,4,machine learning
4,2017-01-05,0.0,6,4,visualization


Shape of the daily DataFrame is (365, 5)


In [90]:
# General data observance
tracking.apply(lambda col: np.sort(col.unique()))

daily.apply(lambda col: np.sort(col.unique()))
# daily.headliner_topic.unique()
daily.headliner_topic = daily.headliner_topic.str.capitalize().replace({'Machine learning': 'ML', 'Ml': 'ML'})

In [91]:
# aggregating the tracking data
tracking = tracking.groupby(['Day']) \
                    .agg({'Duration':'mean', 
                          'gender':'count'}) \
                    .rename(columns = {'Duration':'Duration_Mean', 
                                       'gender':'Num_Visitors'}) \
                    .reset_index()

# add day of the week for both data sets
# daily['DayOfWeek'] = 
tracking['DayOfWeek'] = pd.to_datetime(tracking.Day).dt.day_name()

daily['DayOfWeek'] = daily.date.dt.day_name()

In [104]:
# remove np.nans
tracking.apply(lambda col: col.isna().sum())
daily.apply(lambda col: col.isna().sum())

daily = daily.loc[daily.notna().apply(lambda row: np.prod(row), axis=1).map({1: True, 0: False}),:].reset_index(drop=True)


,date,news,num_ads,num_links,headliner_topic,DayOfWeek
337,2017-12-27,1.0,7,10,Technology,Wednesday
338,2017-12-28,0.0,8,10,Regression,Thursday
339,2017-12-29,1.0,8,10,Regression,Friday
340,2017-12-30,1.0,8,9,Visualization,Saturday
341,2017-12-31,1.0,7,8,Regression,Sunday


In [106]:
display(tracking.head(2))
display(daily.head(2))

,Day,Duration_Mean,Num_Visitors,DayOfWeek
0,2017-03-15,6.589552,134,Wednesday
1,2017-03-16,6.755556,90,Thursday


,date,news,num_ads,num_links,headliner_topic,DayOfWeek
0,2017-01-01,1.0,5,3,Technology,Sunday
1,2017-01-02,1.0,5,1,Technology,Monday


##### 1. Why is it useful information to know that ABC does not plan on having future observations with 70 advertisements?

Removing the observations with 70 ads will be realistic of the future population of ads

##### 2. Should the variable for the number of visits on a day be used in the data set? Why or why not?

This information is not known at the time of prediction. It would be possible to use the number of visits on the previous day to predict the average time on the website, but today's data will not be known.

##### 3. The data ABC got from the tracking company includes the sex of the individual who visited the site. ABC is considering including that in the model somehow, or perhaps building separate models for each sex. Comment on any ethical implications for doing that

In many instances, it would be unethical to break out models based on sex. It is not ethical to attempt and infer someone's sex, but, if this information was known, it could be useful in modeling.